# Libraries

In [1]:
import pandas as pd
import io
import requests
import zipfile
import os

# Extract

In [2]:
response = requests.get('https://bit.ly/416WE1X')
response.raise_for_status() # raise an error for bad status codes

with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
    zip_file.extractall(os.getcwd())

In [3]:
def extract():

    # Read datasets
    dataset1 = pd.read_csv('dataset1.csv')
    dataset2 = pd.read_csv('dataset2.csv')
    dataset3 = pd.read_csv('dataset3.csv')

    # Calculate missing value percentages
    missing1 = dataset1.isnull().mean() * 100
    missing2 = dataset2.isnull().mean() * 100
    missing3 = dataset3.isnull().mean() * 100

    #return data types
    dt_type1 = dataset1.info()
    dt_type2 = dataset2.info()
    dt_type3 = dataset3.info()


    return (dataset1, missing1, dt_type1), (dataset2, missing2,dt_type2), (dataset3, missing3, dt_type3)

In [16]:
(dataset1, missing1, dt_type1), (dataset2, missing2,dt_type2), (dataset3, missing3, dt_type3) = extract()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   customer_id          15 non-null     int64 
 1   date_of_purchase     15 non-null     object
 2   total_amount_billed  15 non-null     int64 
 3   payment_status       15 non-null     object
 4   payment_method       15 non-null     object
 5   promo_code           12 non-null     object
 6   country_of_purchase  15 non-null     object
dtypes: int64(2), object(5)
memory usage: 972.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         15 non-null     int64 
 1   date_of_payment     15 non-null     object
 2   amount_paid         15 non-null     int64 
 3   payment_method      15 non-null     object
 4   payment

We see promo code in dataset1 has 20% missing values so we will replace the missing values with No Promo just incase one would want to analyze customer purchase behaviour.

In [52]:
def transform():

    #copy datasets for transformation
    df1 = dataset1.copy()
    df2 = dataset2.copy()
    df3 = dataset3.copy()

    # replace missing values in promo_code column with No Promo
    # df1.fillna({'promo_code': 'No Promo'}, inplace = True)
    df1['promo_code']= df1['promo_code'].fillna('No Promo')
    

    # change date column datatype to date
    df1['date_of_purchase'] = pd.to_datetime(df1['date_of_purchase'], errors='coerce')
    df2['date_of_payment'] = pd.to_datetime(df2['date_of_payment'], errors='coerce')
    df3['date_of_refund'] = pd.to_datetime(df3['date_of_refund'], errors='coerce')

    #merge datasets
    df1.rename(columns={'country_of_purchase': 'country'}, inplace=True) # first rename country column because they are the same accross
    df2.rename(columns={'country_of_payment': 'country'}, inplace=True) # first rename country column because they are the same accross
    df3.rename(columns={'country_of_refund': 'country'}, inplace=True) # first rename country column because they are the same accross

    # now merge df1 and df2 both on the customer_id and country column and payment_method
    df1and2 = df1.merge(df2,on=['customer_id','country','payment_method'], how ='left') 

    # now merge df1and2 to df3 on the customer_id and country_of_purchase/payment/refund column
    merged_df = df1and2.merge(df3, on = ['customer_id','country'], how='left')

    return merged_df


In [53]:
transform()

,customer_id,date_of_purchase,total_amount_billed,payment_status_x,payment_method,promo_code,country,date_of_payment,amount_paid,payment_status_y,late_payment_fee,date_of_refund,refund_amount,reason_for_refund
0,101,2021-04-01,100,paid,credit card,PROMO1,USA,2021-04-01,100,paid,0,2021-04-03,100,product not as described
1,102,2021-04-02,200,paid,bank transfer,PROMO2,USA,2021-04-03,200,paid,0,2021-04-06,200,defective product
2,103,2021-04-02,50,overdue,credit card,No Promo,UK,2021-04-03,75,paid,10,2021-04-07,75,change of mind
3,104,2021-04-03,75,disputed,e-wallet,PROMO3,UK,2021-04-04,50,overdue,0,2021-04-08,50,product not received
4,105,2021-04-04,125,paid,credit card,PROMO4,USA,2021-04-05,125,paid,0,2021-04-09,25,product not as described
5,106,2021-04-05,150,paid,credit card,No Promo,UK,2021-04-06,150,paid,0,2021-04-11,125,defective product
6,107,2021-04-06,75,overdue,e-wallet,PROMO5,USA,2021-04-07,75,overdue,20,2021-04-12,150,change of mind
7,108,2021-04-06,100,overdue,bank transfer,PROMO6,USA,2021-04-07,100,overdue,30,2021-04-13,75,product not as described
8,109,2021-04-07,50,paid,bank transfer,No Promo,UK,2021-04-08,50,paid,0,2021-04-13,100,defective product
9,110,2021-04-07,25,overdue,credit card,PROMO7,USA,2021-04-08,25,paid,0,2021-04-14,50,product not received
